In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
import re

/var/folders/sc/c5y2x40j3vlctkp8qgp0hmv80000gn/T/ipykernel_20374/2961922991.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
full_df = pd.read_csv("grouped-data.csv")
grouped_df = full_df.iloc[:1000]

/var/folders/sc/c5y2x40j3vlctkp8qgp0hmv80000gn/T/ipykernel_20374/2452374269.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  full_df = pd.read_csv("grouped-data.csv")


In [3]:
print(grouped_df.head(1), "\n")
# print(grouped_df.loc[0, ''])

   Unnamed: 0  User-ID        ISBN Year-Of-Publication         Book-Author  \
0           0        2  0195153448                2002  Mark P. O. Morford   

   Book-Rating  
0          0.0   



In [4]:
# Function to clean text
def clean_text(text):
    # Remove whitespace, punctuation, and convert to lowercase
    cleaned_text = re.sub(r'[^\w\s]', '', text).replace(' ', '').lower()
    return cleaned_text

In [5]:
grouped_df['Book-Author'] = grouped_df['Book-Author'].apply(clean_text)
print(grouped_df.head(5), "\n")

   Unnamed: 0  User-ID        ISBN Year-Of-Publication         Book-Author  \
0           0        2  0195153448                2002       markpomorford   
1           1        8  0002005018                2001  richardbrucewright   
2           2        8  0060973129                1991          carlodeste   
3           3        8  0374157065                1999      ginabarikolata   
4           4        8  0393045218                1999           ejwbarber   

   Book-Rating  
0          0.0  
1          5.0  
2          0.0  
3          0.0  
4          0.0   



/var/folders/sc/c5y2x40j3vlctkp8qgp0hmv80000gn/T/ipykernel_20374/439466849.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grouped_df['Book-Author'] = grouped_df['Book-Author'].apply(clean_text)


In [6]:
grouped_df = pd.get_dummies(grouped_df, columns=['Book-Author'])

In [7]:
X = grouped_df.drop(['User-ID', 'ISBN', 'Book-Rating'], axis=1)
y = grouped_df['Book-Rating']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
print(grouped_df.tail(3))

     Unnamed: 0  User-ID        ISBN Year-Of-Publication  Book-Rating  \
997         997      507  0312986769                2003          9.0   
998         998      507  0312990456                2003          0.0   
999         999      507  0312995431                2004          0.0   

     Book-Author_acgordon  Book-Author_adamlebor  Book-Author_adelelang  \
997                 False                  False                  False   
998                 False                  False                  False   
999                 False                  False                  False   

     Book-Author_adelineyenmah  Book-Author_adhudler  ...  \
997                      False                 False  ...   
998                      False                 False  ...   
999                      False                 False  ...   

     Book-Author_williameisner  Book-Author_williamgeraldgolding  \
997                      False                             False   
998                      

In [17]:
knn_model = NearestNeighbors(n_neighbors=5, metric='euclidean')
knn_model.fit(X_train)

NearestNeighbors(metric='euclidean')

In [18]:
def recommend_books(user_features):
    _, indices = knn_model.kneighbors([user_features])
    return grouped_df.iloc[indices[0]][['ISBN', 'Book-Rating']]

In [21]:
sample_user_features = {
    'Year-Of-Publication': 1995,
    'Book-Author': 'johnsmith',  # Example author
    'Publisher': 'ABC Publishing'  # Example publisher
}

# Preprocess sample user features
sample_author = 'Book-Author_' + sample_user_features['Book-Author'].replace(' ', '_')

In [23]:
# Create feature vector for the sample user
user_features = [
    sample_user_features['Year-Of-Publication'],
    0,  # For Publisher (assuming it's not available in sample user data)
    grouped_df.columns.get_loc(sample_author)
]

KeyError: 'Book-Author_John_Smith'

In [ ]:
# Recommend books for the sample user
recommended_books = recommend_books(user_features)
print("Recommended Books:")
print(recommended_books)